In [1]:
import sys
import subprocess
sys.path.append('f:/seismic/code/core')
sys.path.append('/seismic/code/core')
sys.path.append('/kaggle/input/my-seismic-library/')
import kaggle_support as kgs
kgs.preallocate_matrices=False
import importlib
import seis_model
import time
import pandas as pd
import numpy as np
#kgs.cache_dir_write = kgs.cache_dir_write+'/../cache_evaluate/'
#kgs.cache_dir_read = kgs.cache_dir_write

F:/seismic/models/brendan/*.pth
Loading:  F:/seismic/models/brendan\unet2d_caformer_seed3_epochbest.pt
n_models: 1


In [2]:
# Set up model
importlib.reload(seis_model)
model = seis_model.default_model()
model.models[1].model_FlatVel = seis_model.DummyModel()
model.models[1].model_FlatVel.state=1
model.models[1].model_Style_A = seis_model.DummyModel()
model.models[1].model_Style_A.state=1
model.models[1].model_TV2D = seis_model.DummyModel()
model.models[1].model_TV2D.state=1
model.read_cache = False

In [3]:
%%time
data = kgs.load_all_train_data(validation_only=True)[::25]
model.models[1].run_in_parallel = True
data_out = model.infer(data)

Processing in parallel : 100%|███████████████████████████████████████████████████████| 200/200 [00:41<00:00,  4.86it/s]


CPU times: total: 2.7 s
Wall time: 46.1 s


In [4]:
dfTimings = pd.read_csv(kgs.timing_filename)
dfTimings = dfTimings.groupby('family')['time_taken'].mean().to_frame('time_taken')

In [5]:
#kgs.disable_caching = True
families = list(set([d.family for d in data]))
families.sort()
res = dict()
res['family'], res['score_brendan'], res['score_mine'], res['score_mine_max'], res['time_taken'] = [],[],[],[],[]
for f in families:
    res['family'].append(f)
    this_data_out = []
    for d in data_out:
        if d.family==f:
            this_data_out.append(d)
    data_out_brendan = model.models[0].infer(this_data_out)
    res['score_brendan'].append(kgs.score_metric(data_out_brendan,show_diagnostics=False)[0])
    #data_out = model.infer(this_data[:1])
    t=time.time()
    try:
        res['time_taken'].append(dfTimings.loc[f].to_numpy()[0])
    except:
        res['time_taken'].append(0)
    score,_,res_all = kgs.score_metric(this_data_out,show_diagnostics=False)
    res['score_mine'].append(score)
    res['score_mine_max'].append(np.max(res_all))
    #print(pd.DataFrame(res))
df = pd.DataFrame(res)
print('')
print('')
print('Brendan local score: ', np.mean(df['score_brendan']))
print('My local score: ', np.mean(df['score_mine']))
print('Estimate public LB: ', 28.8*np.mean(df['score_mine'])/np.mean(df['score_brendan']))
df

Importing cache brendan: 100%|███████████████████████████████████████████████████████| 20/20 [00:00<00:00, 9948.54it/s]



Brendan local score:  25.6438103677555
My local score:  26.192869116958303
Estimate public LB:  29.41663581777706


,family,score_brendan,score_mine,score_mine_max,time_taken
0,CurveFault_A,3.507875,3.507875,6.833073,0.750085
1,CurveFault_B,78.828673,78.828673,151.928980,0.212596
2,CurveVel_A,8.347847,8.347847,15.502449,0.176444
3,CurveVel_B,41.790949,41.790949,174.873469,0.205876
4,FlatFault_A,2.964055,2.964055,20.262268,0.216852
5,FlatFault_B,28.442969,28.442969,105.244286,0.195994
6,FlatVel_A,1.069388,1.069388,2.304898,0.008951
7,FlatVel_B,8.323959,8.323959,56.527755,0.009185
8,Style_A,33.722460,33.722460,92.180529,0.029228
9,Style_B,49.439928,54.930515,86.619562,0.036548


In [6]:
# No extra model: 28.8
# FlatVel:
# StyleA: 

In [7]:
28-30*0.081

25.57